In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

# Load datasets
train_file = 'data/data_latih.csv'  # Training dataset
test_file = 'data/data_uji.csv'  # Testing dataset

df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

# Extract features and labels from training data
features = ['Area', 'Parameter', 'Aspect Ratio', 'Solidity']
x_train = df_train[features].values
y_train = df_train['Label'].values

# Encode species labels to integers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

# Normalize feature values
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(df_test[features].values)

# Define the neural network structure
n_inputs = len(features)
n_hidden = 5
n_outputs = len(set(y_train))
l_rate = 0.3
n_epoch = 500

# Initialize network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = [
        [{'weights': np.random.rand(n_inputs + 1)} for _ in range(n_hidden)],
        [{'weights': np.random.rand(n_hidden + 1)} for _ in range(n_outputs)]
    ]
    return network

# Activation function
def sigmoid(x, derivative=False):
    if derivative:
        return x * (1.0 - x)
    return 1.0 / (1.0 + np.exp(-x))

# Forward propagation
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = neuron['weights'][-1] + np.dot(neuron['weights'][:-1], inputs)
            neuron['output'] = sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Backward propagation of error
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = sum([neuron['weights'][j] * neuron['delta'] for neuron in network[i + 1]])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid(neuron['output'], derivative=True)

# Update weights
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1] if i == 0 else [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train the network
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row[:-1])
            expected = [0 for _ in range(n_outputs)]
            expected[int(row[-1])] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

# Make a prediction
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Train the model
train_data = np.column_stack((x_train, y_train.astype(int)))
network = initialize_network(n_inputs, n_hidden, n_outputs)
train_network(network, train_data, l_rate, n_epoch, n_outputs)

# Predict on test data
predictions = [predict(network, row) for row in x_test]
predicted_species = label_encoder.inverse_transform(predictions)

# Save predictions
df_test['Predicted_Species'] = predicted_species
df_test.to_csv('data/predictions.csv', index=False)

# Optional: plot training accuracy if needed
print("Predictions saved to 'predictions.csv'")

Predictions saved to 'predictions.csv'


In [5]:
import pandas as pd

# Load the actual output and predicted output files
actual_file = 'data/data_uji_asli.csv'  # File dengan label asli
predicted_file = 'data/predictions.csv'  # File dengan prediksi

df_actual = pd.read_csv(actual_file)
df_predicted = pd.read_csv(predicted_file)

# Extract the columns of interest
actual_labels = df_actual['Label']
predicted_labels = df_predicted['Predicted_Species']

# Compare actual and predicted labels
correct_predictions = (actual_labels == predicted_labels).sum()
total_predictions = len(actual_labels)

# Calculate accuracy
accuracy = (correct_predictions / total_predictions) * 100

print(f'Tingkat keakuratan metode klasifikasi: {accuracy:.2f}%')


Tingkat keakuratan metode klasifikasi: 75.00%
